##### Importing libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score,roc_auc_score

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/entbappy/Branching-tutorial/refs/heads/master/tweet_emotions.csv')

In [3]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
df.shape

(40000, 3)

In [5]:
df.drop(columns=['tweet_id'], inplace=True)

In [6]:
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [7]:
# to simplyfies the project instead of 3 labels only 2 labels we are consider

final_df = df[df['sentiment'].isin(['happiness', 'sadness'])]


In [8]:
final_df

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
6,sadness,"I should be sleep, but im not! thinking about ..."
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?
...,...,...
39986,happiness,going to watch boy in the striped pj's hope i ...
39987,happiness,"gave the bikes a thorough wash, degrease it an..."
39988,happiness,"had SUCH and AMAZING time last night, McFly we..."
39994,happiness,Succesfully following Tayla!!


In [9]:
final_df.sample(5)

,sentiment,content
16439,happiness,Six feet under theme is on jjj
4177,sadness,@mgiraudofficial i'd give anything to be ther...
37867,happiness,"@noxhanti LOL --&gt; don't mention it, my ple..."
19608,sadness,Last full day of high school was today. I'm sa...
33943,happiness,"Is finally back from the dinner party, had fun..."


In [10]:
final_df.shape

(10374, 2)

### To covert the happiness label into 1 and sadness into 2

In [11]:
final_df['sentiment'].replace({'happiness':1, 'sadness':0}, inplace=True)

C:\Users\ell.DESKTOP-F8FOOJM\AppData\Local\Temp\ipykernel_17020\618365853.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['sentiment'].replace({'happiness':1, 'sadness':0}, inplace=True)
C:\Users\ell.DESKTOP-F8FOOJM\AppData\Local\Temp\ipykernel_17020\618365853.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [12]:
final_df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [13]:
#train test operation

train_data, test_data = train_test_split(final_df, test_size=0.2, random_state = 42 )

### Data Preprocessing

In [14]:
nltk.download('wordnet')
nltk.download('stopwords')

def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)


def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    text = [i for i in str(text).split() if i not in stop_words]
    return " ".join(text)

def removing_numbers(text): 
    text = ''.join(i for i in text if not i.isdigit())
    return text

def lower_case(text):
    text = text.split()
    
    text = [y.lower() for y in text]
    return " ".join(text)

def removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df):
    df.content=df.content.apply(lambda content : lower_case(content))
    df.content=df.content.apply(lambda content : remove_stop_words(content))
    df.content=df.content.apply(lambda content : removing_numbers(content))
    df.content=df.content.apply(lambda content : removing_punctuations(content))
    df.content=df.content.apply(lambda content : removing_urls(content))
    df.content=df.content.apply(lambda content : lemmatization(content))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= removing_numbers(sentence)
    sentence= removing_punctuations(sentence)
    sentence= removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ell.DESKTOP-
[nltk_data]     F8FOOJM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ell.DESKTOP-
[nltk_data]     F8FOOJM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
normalized_sentence("That's it? It's done already? This is one")

'that s it done already one'

In [16]:
train_data = normalize_text(train_data)
test_data = normalize_text(test_data)

In [17]:
train_data

,sentiment,content
23531,0,quot my problem miss you cause don t quot
8051,0,that s it done already one proof there s nothi...
11499,0,hungry food steal
31288,1,foot hurt finally bed will forget crunch over ...
18561,0,really ill atm
...,...,...
21697,1,chocolatesuze yes yes should especially wine m...
19445,0,kickzfadayz boy better get tonight
20216,1,tafe actually quite good
3258,0,minute boarding hour home window seat


### Feature Engineering

In [18]:
x_train = train_data['content'].values
y_train = train_data['sentiment'].values

x_test = test_data['content'].values
y_test = test_data['sentiment'].values

In [19]:
#Apply Bag of Words (CountVectorizer)

vectorizer = CountVectorizer()

x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.transform(x_test)

In [20]:
train_df = pd.DataFrame(x_train_bow.toarray())

train_df['label'] =y_train
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14223,14224,14225,14226,14227,14228,14229,14230,14231,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Model Building

In [23]:
xgb_model = xgb.XGBClassifier(use_label_encoder = False, eval_metrics= 'mlogloss')

xgb_model.fit(x_train_bow, y_train)

y_pred = xgb_model.predict(x_test_bow)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy: ",accuracy)
print("classification_report: \n",classification_rep)

c:\Users\ell.DESKTOP-F8FOOJM\anaconda3\envs\test\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:08:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "eval_metrics", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy:  0.771566265060241
classification_report: 
               precision    recall  f1-score   support

           0       0.75      0.83      0.79      1060
           1       0.80      0.71      0.75      1015

    accuracy                           0.77      2075
   macro avg       0.77      0.77      0.77      2075
weighted avg       0.77      0.77      0.77      2075



### Model Evaluation

In [25]:
y_pred = xgb_model.predict(x_test_bow)
y_pred_proba = xgb_model.predict_proba(x_test_bow)[:,1]



precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

In [26]:
print(f"precision: {precision}")
print(f"Recall: {recall}")
print(f"AUC: {auc}")

precision: 0.7988950276243094
Recall: 0.7123152709359606
AUC: 0.7703085788642067
